# Credit Card Fraud Detection

In [31]:
from IPython.display import display, HTML
import requests
import re
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import seaborn as sns
from PIL import Image
from io import BytesIO
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, precision_recall_fscore_support

In [6]:
# URL of the Mercedes-Benz logo
image_url = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSOBStjozp5ASwmTNTQxH7TqpGfhOIlyDClVQ&s"

# Fetch the image from the URL
response = requests.get(image_url)

# Check if the response content type is an image
if 'image' in response.headers.get('content-type', ''):
    # Open the image using PIL
    image = Image.open(BytesIO(response.content))
    
    # Save the image to a temporary file
    temp_image_path = "temp_image.png"
    image.save(temp_image_path)

    # Use Google Fonts for script font
    google_fonts_link = '<link href="https://fonts.googleapis.com/css2?family=Great+Vibes&display=swap" rel="stylesheet">'
    
    # Display the image and text side by side using HTML
    html = f"""
    {google_fonts_link}
    
    <div style="display: flex; align-items: center; padding: 20px;">
        <img src="{temp_image_path}" style="width: 400px; height: auto; border: 2px solid #000; border-radius: 10px;"/>
        <div style="margin-left: 30px; font-family: 'Arial', sans-serif; color: #333;">
        
            <h2 style="
                font-size: px; 
                font-weight: bold; 
                color: transparent; 
                background: linear-gradient(90deg, #8A2BE2, #1E90FF, #FFD700); 
                -webkit-background-clip: text; 
                background-clip: text; 
                font-family: 'Sacramento', cursive;">
                Credit Card Fruad Detection 
            </h2>
            <p style="font-size: 16px; line-height: 1.6; margin-top: 10px;">
            <h3>Introduction</h3>
In September 2013, a dataset of credit card transactions was collected from European cardholders, comprising a total of 284,807 transactions. This dataset is notably imbalanced, with only 492 transactions identified as fraudulent, accounting for just 0.172% of the total. The transactions include numerical input variables that have been transformed using Principal Component Analysis (PCA) to protect confidentiality. This transformation results in the features V1 through V28. The dataset also includes two non-transformed features: Time, representing the seconds elapsed since the first transaction, and Amount, the transaction amount. The Class feature denotes whether a transaction is fraudulent (1) or not (0).
<h3> Problem Formulation</h3>
The primary challenge in this dataset is the detection of fraudulent transactions amidst a vast majority of non-fraudulent ones. Given the extreme class imbalance, traditional classification methods may struggle to accurately identify the minority class (frauds). The problem is further complicated by the fact that the original features have been transformed and are not available for direct interpretation. Thus, the focus is on developing and evaluating methods to effectively detect fraudulent transactions using the transformed principal component features, Time, and Amount.
<h3>Aim</h3>
The aim of this project is to develop and implement a robust machine learning model capable of accurately identifying fraudulent transactions within the credit card transaction dataset. This involves handling the class imbalance effectively and utilizing the transformed features, Time, and Amount to build a predictive model that can distinguish between fraudulent and non-fraudulent transactions.
            </p>
        </div>
    </div>
    """

    display(HTML(html))
else:
    print("The URL does not point to an image file.")


# Loading Dataset

In [7]:
df = pd.read_csv('creditcard.csv')

In [8]:
df.head()

Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28  Amount  Class  
0 -0.189115  0.133558 -0.021053  149.62      0  
1  0.125895 -0.008983  0.014724    2.69      0  
2 -0.139097 -0.055353 -0.059752  378.66      0  
3 -0.221929  0.062723  0.061458  123.50      0  
4  0.502292  0.219422  0.215153   69.99      0  

[5 rows x 31 columns]

# 🔍 Exploratory Data Analysis

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [11]:
# Missing Values
df.isnull().sum().sum()

0

In [12]:
df.columns

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')

In [15]:
# Normalize the 'Amount' and 'Time' columns
scaler = StandardScaler()
df[['Amount', 'Time']] = scaler.fit_transform(df[['Amount', 'Time']])


In [16]:
# Separate features and target
X = df.drop(columns='Class')
y = df['Class']

In [18]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=123)

In [23]:
# Logistic Regression model
log_reg = LogisticRegression(random_state=123)
log_reg.fit(X_train, y_train)

LogisticRegression(random_state=123)

In [26]:
# Make predictions
y_pred_log_reg = log_reg.predict(X_test)


# Evaluate Logistic Regression model
print("Logistic Regression:")
print(classification_report(y_test, y_pred_log_reg))



Logistic Regression:
              precision    recall  f1-score   support

           0       0.92      0.98      0.95     56747
           1       0.97      0.92      0.95     56979

    accuracy                           0.95    113726
   macro avg       0.95      0.95      0.95    113726
weighted avg       0.95      0.95      0.95    113726



In [32]:
# Calculate and round the correlation matrix
corr = df.corr().round(1)

# Create the heatmap using plotly.express
fig = px.imshow(
    corr, 
    color_continuous_scale='Viridis'  # Choose a color scale for the heatmap
)

# Update the layout with customized styles
fig.update_layout(
    title_text="Credit Card Fraud Correlation Matrix",
    title_font_color="white",
    title_font_size=18,  # Set title font size
    legend_title_font_color="yellow",
    paper_bgcolor="black",  # Set background color of the entire plot
    plot_bgcolor='grey',  # Set background color of the plot area
    font=dict(
        color="white",  # Set font color for text
        family="Arial",  # Set font family
        size=12  # Set font size
    ),
    autosize=False,  # Disable automatic sizing
    width=800,  # Set the width of the figure to be small
    height=800  # Set the height of the figure to be small
)

# Show the plot
fig.show()


In [35]:

# Create the density plot using plotly.express
fig = px.histogram(
    df, 
    x='Time', 
    color='Class',
    histnorm='density',  # Normalize the histogram to show density
    labels={'Time': 'Time [s]', 'Class': 'Transaction Class'},
    title='Credit Card Transactions Time Density Plot',
    opacity=0.75  # Slight transparency for better overlay visibility
)

# Customize the layout to enable cursor interactions
fig.update_layout(
    xaxis_title='Time [s]',
    yaxis_title='Density',
    title_font_size=16,
    xaxis_title_font_size=14,
    yaxis_title_font_size=14,
    plot_bgcolor='white',  # Set background color to white
    paper_bgcolor='lightgrey',  # Set paper background color to light grey
    margin=dict(l=40, r=40, t=60, b=40),  # Adjust margins for better spacing
    hovermode='x unified',  # Enable unified hover mode
)

# Show the plot
fig.show()


In [38]:
import plotly.express as px

# Count the occurrences of each class
class_counts = df["Class"].value_counts().reset_index()
class_counts.columns = ['Class', 'Number of Transactions']

# Create the bar chart using plotly.express
fig = px.bar(
    class_counts, 
    x='Class', 
    y='Number of Transactions',
    color='Class',
    color_discrete_map={0: 'blue', 1: 'red'},  # Assign colors for each class
    text='Number of Transactions',
    labels={'Class': 'Class', 'Number of Transactions': 'Number of Transactions'},
    title='Credit Card Fraud Class Distribution (0 = Not Fraud, 1 = Fraud)'
)

# Customize the layout
fig.update_layout(
    xaxis_title='Class',
    yaxis_title='Number of Transactions',
    title_font_size=16,
    xaxis_title_font_size=14,
    yaxis_title_font_size=14,
    legend_title='Class',
    legend_title_font_size=12,
    legend_font_size=12,
    plot_bgcolor='white',  # Set background color to white
    paper_bgcolor='lightgrey',  # Set paper background color to light grey
    margin=dict(l=40, r=40, t=60, b=40)  # Adjust margins for better spacing
)

# Show the plot
fig.show()
